In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
cd gdrive/My Drive/Voice_Cloning

/content/gdrive/My Drive/Voice_Cloning


In [5]:
cd autovc

/content/gdrive/My Drive/Voice_Cloning/autovc


In [10]:
import numpy as np
from math import ceil
from model_vc import Generator


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

# device = 'cuda:0'
# G = Generator(32,256,512,32).eval().to(device)

# g_checkpoint = torch.load('autovc.ckpt')
# G.load_state_dict(g_checkpoint['model'])

metadata = pickle.load(open('metadata.pkl', "rb"))
print(metadata)
spect_vc = []

for sbmt_i in metadata:
             
    x_org = sbmt_i[2]
    x_org, len_pad = pad_seq(x_org)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata:
                   
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        
        
with open('results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)

[['p225', array([-2.92885415e-02,  1.67739280e-02,  6.42375797e-02,  5.12384847e-02,
        8.77934247e-02, -4.05867286e-02,  1.08849108e-02,  2.06673276e-02,
       -8.39690343e-02,  1.48199843e-02,  3.82344425e-02,  1.12243919e-02,
        4.62971367e-02,  1.66661311e-02, -5.78785129e-02,  3.60288732e-02,
        1.92339886e-02,  9.17971320e-03,  2.74707917e-02, -5.48039749e-02,
        2.50798557e-02,  4.66737375e-02, -6.14981353e-03,  3.88026945e-02,
        5.68139665e-02, -7.33052716e-02,  2.30920967e-02, -1.04292825e-01,
       -2.61898227e-02,  3.02257240e-02, -3.02889403e-02,  3.63447554e-02,
       -4.97230627e-02,  1.57715172e-01, -3.93295921e-02,  5.51161245e-02,
        4.64604087e-02, -4.59927395e-02, -4.82378080e-02, -3.81431282e-02,
        3.91379185e-02,  4.59317304e-02, -1.55072343e-02, -1.67513415e-02,
       -5.09507731e-02, -5.08496165e-02, -2.25679334e-02,  2.63878461e-02,
       -1.26611767e-02, -2.49883570e-02,  3.02621610e-02, -2.48487398e-04,
       -1.22652

In [2]:
!ls

sample_data


In [8]:

import torch
import librosa
import pickle
from synthesis import build_model
from synthesis import wavegen

spect_vc = pickle.load(open('results.pkl', 'rb'))
device = torch.device("cuda")
model = build_model().to(device)
checkpoint = torch.load("checkpoint_step001000000_ema.pth")
model.load_state_dict(checkpoint["state_dict"])

for spect in spect_vc:
    name = spect[0]
    c = spect[1]
    print(name)
    print(c)
    # waveform = wavegen(model, c=c)   
    # librosa.output.write_wav(name+'.wav', waveform, sr=16000)

p225xp225
[[0.38280225 0.32192802 0.27241024 ... 0.18827914 0.18116063 0.17475641]
 [0.37740543 0.30501315 0.31424838 ... 0.23313928 0.18586273 0.1498867 ]
 [0.3885517  0.28357577 0.2739416  ... 0.13206439 0.10279441 0.10369424]
 ...
 [0.57143533 0.6357635  0.5616603  ... 0.21222183 0.19688344 0.18200691]
 [0.5051673  0.5022582  0.4819557  ... 0.18236059 0.16847152 0.15469944]
 [0.4489379  0.44055852 0.46094942 ... 0.14624405 0.13292265 0.13138223]]
p225xp228
[[0.43724388 0.35739926 0.30414346 ... 0.20524152 0.19167462 0.17206721]
 [0.4447977  0.35146433 0.3134007  ... 0.23535398 0.23473118 0.22268611]
 [0.4352066  0.3636808  0.34444585 ... 0.20809732 0.19319981 0.18029554]
 ...
 [0.47883365 0.4589784  0.49727875 ... 0.22472677 0.22585846 0.2406255 ]
 [0.5128157  0.4635     0.47831792 ... 0.19811438 0.1886824  0.19102778]
 [0.52412266 0.45657018 0.45981112 ... 0.16978851 0.15388669 0.14956051]]
p225xp256
[[0.41864657 0.31885418 0.27399325 ... 0.2314785  0.22384298 0.22106947]
 [0.43175

In [7]:
!pip3 install wavenet_vocoder

  Created wheel for wavenet-vocoder: filename=wavenet_vocoder-0.1.1-cp36-none-any.whl size=12667 sha256=23b0726531cf2959ff4522fd6e5568302ddb9dd402d485ab28cbab202adcbd0c
  Stored in directory: /root/.cache/pip/wheels/72/fc/21/02d3785b65dd072b110b44b9df98b8cbf72a89ddea424ff0d9
Successfully built wavenet-vocoder
